In [1]:
from vnstock import *
import yfinance as yf
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import Ridge
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
data_ticker = pd.read_excel("C:\\Users\\Chi Bao\\OneDrive\\Desktop\\Đồ án tốt nghiệp\\data_ticker.xlsx")

FI = ['GC=F','SI=F','CL=F','^GSPC','^DJI','^IXIC','^RUT','^FTSE','^N225','^NYA']
df = pd.DataFrame()
for symbol in FI:
    data = yf.download(symbol, start='2012-03-20', end='2023-09-06')
    df[symbol] = data['Adj Close']

column_mapping = {
    'GC=F': 'Gold',
    'SI=F': 'Silver',
    'CL=F': 'Crude Oil',
    '^GSPC': 'S&P 500',
    '^DJI': 'Dow Jones',
    '^IXIC': 'NASDAQ',
    '^RUT': 'Russell 2000',
    '^FTSE': 'FTSE 100',
    '^N225': 'Nikkei 225',
    '^NYA': 'NYSE'
}

df = df.rename(columns=column_mapping)
df.dropna()

returns_df = df.pct_change()*100
returns_df.columns = [col + '_return' for col in returns_df.columns]
returns_df = returns_df.dropna()

###

FI = ['VNINDEX','HNX','VN30','HNX30','UPCOM']
data = stock_historical_data('VNINDEX', '2012-03-19', '2023-09-05', "1D", "index")
d_new = data.set_index('time')
d_new['VNINDEX'] = data['close']
df1 = pd.DataFrame()
for symbol in FI:
    data = stock_historical_data(symbol, '2012-03-19', '2023-09-05', "1D", "index")
    data = data.set_index('time')
    df1[symbol] = data['close']
df1 = df1.dropna()

returns_df1 = df1.pct_change()*100
returns_df1.columns = [col + '_return' for col in returns_df1.columns]
returns_df1 = returns_df1.dropna()
returns_df1.head(42069)

###

lagreturn = pd.DataFrame()

for column in returns_df.columns:
    lagreturn[f'{column}_Lag1'] = returns_df[column].shift(1)
for column in returns_df.columns:
    lagreturn[f'{column}_Lag2'] = returns_df[column].shift(2)
lagreturn = lagreturn.dropna()

lagreturn1 = pd.DataFrame()
for column in returns_df1.columns:
    lagreturn1[f'{column}_Lag1'] = returns_df1[column].shift(1)
for column in returns_df1.columns:
    lagreturn1[f'{column}_Lag2'] = returns_df1[column].shift(2)
lagreturn1 = lagreturn1.dropna()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [3]:
def do_tre_vnstock(symbol, start_date, end_date, lags=10): 
    ts = stock_historical_data(symbol,start_date, end_date)
    ts = ts.dropna(subset=['close'])


    tsret = pd.DataFrame(index=ts.index)
    tsret['time']=ts['time']
    tsret["Today"] = ts["close"].pct_change()*100

    for i,x in enumerate(tsret["Today"]):
        if (abs(x) < 0.0001):
            tsret["Today"][i] = 0.0001

    for i in range(0, lags):
        tsret["Lag%s" % str(i+1)] = tsret["Today"].shift(i+1)

    tsret["Direction"] = np.sign(tsret["Today"])
    tsret = tsret.dropna()
    tsret = tsret.set_index('time')
    bpt = pd.merge(tsret, lagreturn, left_index=True, right_index=True)
    bpt1 = pd.merge(bpt, lagreturn1, left_index=True, right_index=True)
    bpt1.fillna(0, inplace=True)
    bpt1 = bpt1[(bpt1 != 0).all(axis=1)]
    return bpt1

In [4]:
#MODEL
list_i = []
list_return_price = []
list_rmse, list_mse, list_mae = [], [], []
list_k = []
list_r2_train = []
list_r2_test = []

for i in data_ticker['ticker']:
  try:
    data = do_tre_vnstock(i, '2012-03-19', '2023-09-05', lags=10)

    X = data.loc[:, ~data.columns.isin(['Today'])]
    y = data["Today"]

    list_k1=[]
    list_score=[]
    for k in np.arange(0.1,0.5,0.01):
      model = Ridge(alpha=1.0)
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = k, random_state = 12, shuffle=False)
      model.fit(X_train, y_train)
      yhat_test = model.predict(X_test)
      score = model.score(X_test, y_test)
      list_k1.append(k)
      list_score.append(score)

    score_max = max(list_score)
    a = list_score.index(score_max)
    k = list_k1[a]

    model = Ridge(alpha=1.0)
    X_train, X_test,y_train, y_test=train_test_split(X,y,test_size=k,random_state=12, shuffle=False)
    model.fit(X_train,y_train)
    yhat_test = model.predict(X_test)
    r2_train = model.score(X_train, y_train)
    r2_test = model.score(X_test, y_test)
    mse = mean_squared_error(y_test, yhat_test, squared=True)
    rmse = mean_squared_error(y_test, yhat_test, squared=False)
    mae = mean_absolute_error(y_test, yhat_test)

    X_predict = [X.iloc[-1].values.tolist()]
    Y_predict = model.predict(X_predict)

    list_i.append(i)
    list_k.append(k)
    list_return_price.append(Y_predict)
    list_r2_train.append(r2_train)
    list_r2_test.append(r2_test)
    list_rmse.append(rmse)
    list_mse.append(mse)
    list_mae.append(mae)

    print('Stock:',i,f' Predict: {Y_predict.item():.2f}',f' R2_train: {r2_train:.2f}',f' R2_test: {r2_test:.2f}',
          f' RMSE: {rmse:.2f}',f' MSE: {mse:.2f}', f'MAE: {mae:.2f}',f' test size: {k:.2f}',)
  except Exception as e:
    print(f"Stock: {i} -Unpredictable!\n" f'{e}')
    continue

Stock: SSI  Predict: -3.01  R2_train: 0.58  R2_test: 0.56  RMSE: 1.88  MSE: 3.54 MAE: 1.49  test size: 0.33
Stock: BCM  Predict: -1.19  R2_train: 0.44  R2_test: 0.27  RMSE: 1.21  MSE: 1.47 MAE: 0.99  test size: 0.10
Stock: VHM  Predict: 1.76  R2_train: 0.52  R2_test: 0.54  RMSE: 1.38  MSE: 1.90 MAE: 1.01  test size: 0.14
Stock: VIC  Predict: 1.22  R2_train: 0.48  R2_test: 0.41  RMSE: 1.77  MSE: 3.15 MAE: 1.18  test size: 0.36
Stock: VRE  Predict: -1.66  R2_train: 0.56  R2_test: 0.46  RMSE: 1.32  MSE: 1.74 MAE: 1.08  test size: 0.20
Stock: BVH  Predict: 1.58  R2_train: 0.63  R2_test: 0.42  RMSE: 1.65  MSE: 2.71 MAE: 1.22  test size: 0.49
Stock: POW  Predict: 1.62  R2_train: 0.57  R2_test: 0.52  RMSE: 1.54  MSE: 2.36 MAE: 1.23  test size: 0.31
Stock: GAS  Predict: 1.55  R2_train: 0.57  R2_test: 0.45  RMSE: 1.51  MSE: 2.27 MAE: 1.09  test size: 0.43
Stock: ACB  Predict: 1.44  R2_train: 0.52  R2_test: 0.45  RMSE: 1.46  MSE: 2.14 MAE: 1.06  test size: 0.46
Stock: BID  Predict: 1.65  R2_trai

In [5]:
KQ = pd.DataFrame({'Stock': list_i, 'Predict': list_return_price, 'R2_train': list_r2_train,'R2_test': list_r2_test,
                    'RMSE': list_mse,'MSE': list_rmse, 'MAE': list_mae,'test size': list_k})
KQ.to_excel("Kết quả linh hoạt test size.xlsx")
KQ.head(42069)

,Stock,Predict,R2_train,R2_test,RMSE,MSE,MAE,test size
0,SSI,[-3.0077553162042197],0.581335,0.555833,3.544090,1.882575,1.486715,0.33
1,BCM,[-1.1888988073339037],0.436066,0.270241,1.465036,1.210387,0.993072,0.10
2,VHM,[1.7626620348329434],0.518986,0.541314,1.901145,1.378820,1.011746,0.14
3,VIC,[1.2153954510482605],0.481600,0.413822,3.146430,1.773818,1.179557,0.36
4,VRE,[-1.659082403986465],0.555930,0.457924,1.741659,1.319719,1.083937,0.20
...,...,...,...,...,...,...,...,...
389,TDP,[0.2869163745891695],0.352569,0.408065,4.979919,2.231573,1.944691,0.10
390,NO1,[-0.18204060062322752],0.579773,0.414386,7.260291,2.694493,1.987703,0.12
391,L10,[2.288937073282475],0.641095,0.628018,11.080857,3.328792,2.724326,0.14
392,REE,[1.3885306792499545],0.495840,0.446846,3.140803,1.772231,1.281520,0.42
